In [37]:
# gerekli kutuphaneler indirilir

import os
from glob import glob
import cv2
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [38]:
template_folder = r"C:\Users\43117073062\Downloads\dataset\dataset\template"
target_folder = r"C:\Users\43117073062\Downloads\dataset\dataset\target"

# pathler üzerinde gezip resimleri etiketleri ile beraber tuttum list veya dictionary yardımıyla
file_list = glob(os.path.join(template_folder, "*.png"))
template_image_name = []
for file_path in file_list:
    file_name = os.path.basename(file_path)
    template_image_name.append(file_name[0])
    
#target folderinda yer alan herbir harf dosyasinin icindeki resimler dictionary yardimiyla tutulur (alphabet_name:images) seklinde
images = {}
for root, dirs, files in os.walk(target_folder):
    for dir in dirs:
        images[dir] = []
    for file in files:
        if file.endswith('.jpg'):
            file_path = os.path.join(root, file)
            parent_dir = os.path.basename(root)
            images[parent_dir].append(file_path)



In [50]:

#en iyi accuracye sahip harf dosyasını gostermek icin degisken tanımladık
accuracy_class_score = {} 

#target image dosyasinda yer alan her bir harf dosyasinin icine giriyoruz
for dir, image_paths in images.items():
    #harf dosyasının adı bızım true_classımızdır
    true_class = dir
    
    #accuracy hesaplarken correct / total ile hesaplama yapacagız 
    total = len(images[dir])
    correct = 0
    
    #bır harf dosyasında en fazla hangı harf ile eslesme yapılmıs onu gostermek ıcın
    folder_best_class = []    
    
    #target_images ta yer alan bır harf dosyasındakı resımlerın üzerinden gezmeye aıt for loop
    for image_path in image_paths:
        target = cv2.imread(image_path) #cv2.IMREAD_GRAYSCALE)
        
        #targetta yer alan resım okundu ve min_max normalizatıon 
        target = cv2.normalize(target, None, 0, 1, cv2.NORM_MINMAX)
        
        # targettaki harf dosyasında yer alan her bir harfin templatte yeralan harflerle olan skorlarını tutucagız ve ardında
        # max degerını tutmak ıcın bır dictionary tanımladım
        scores = {}
        
        #template resimleri uzerinde geziyoruz.
        for template_name in template_image_name:
            template_path = os.path.join(template_folder,template_name+'.png')
            template = cv2.imread(template_path)#, cv2.IMREAD_GRAYSCALE)
            #min-max ile normalize ettik
            template = cv2.normalize(template, None, 0, 1, cv2.NORM_MINMAX)
            #template ve target resımlerının correlation coefficient degerini match_template fonksiyonu ile bulduk
            result = cv2.matchTemplate(target, template, cv2.TM_CCOEFF_NORMED)
            
            #en yüksek sinyal veren yere ait max_value degerini minMaxLoc fonksiyonu ile bulduk
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
            
            #Degerleri score degiskeninde key olarak templatteki harfin adi karsısında en yüksek frekansı tuttuk
            scores[template_name] = max_val
            
        #Enyüksek scora sahıp etiketi buluyoruz ve best_class olarak sakladık
        #Eger best_class true_class a esitse correct sayısını bir arttır
        best_class = max(scores, key=scores.get) #scores degerin maksina ait etiketi bulup best_class olarak sakladık
        
        if true_class == best_class:
            correct += 1
        
        #herbir target image hangi harf olarak etiketlendiklerini yazdıralım
        print("Target image:", os.path.basename(image_path), "is classified as:", best_class,"with best max_value score:", scores[best_class])
        folder_best_class.append(best_class)
           
        
    accuracy = correct / total
    accuracy_class_score[true_class] = accuracy
    
    #burada da harf dosyasına ait accuracy yazdırdık
    print("Accuracy of ",true_class, "is: ", accuracy)
    #burada ayrıca bır harf dosyasındakı 30 resim en cok hangi harf ile eslesmis onu gosterdik
    print(true_class, 'most matched alphabet is', max(set(folder_best_class), key = folder_best_class.count))
    
#en iyi accuracye sahip harf dosyasını gosterdik  
best_accuracy_class = max(accuracy_class_score, key = accuracy_class_score.get)
print('The best classified alphabet is:',best_accuracy_class, 'with score of ', accuracy_class_score[best_accuracy_class] )




Target image: A1.jpg is classified as: X with best max_value score: 0.6113331913948059
Target image: A1001.jpg is classified as: J with best max_value score: 0.41084495186805725
Target image: A101.jpg is classified as: Z with best max_value score: 0.6178134083747864
Target image: A1101.jpg is classified as: B with best max_value score: 0.47016027569770813
Target image: A1201.jpg is classified as: X with best max_value score: 0.6400721669197083
Target image: A1301.jpg is classified as: N with best max_value score: 0.36061611771583557
Target image: A1401.jpg is classified as: Z with best max_value score: 0.7446790337562561
Target image: A1501.jpg is classified as: N with best max_value score: 0.36551231145858765
Target image: A1601.jpg is classified as: S with best max_value score: 0.45880091190338135
Target image: A1701.jpg is classified as: X with best max_value score: 0.7375814914703369
Target image: A1801.jpg is classified as: C with best max_value score: 0.1560995727777481
Target im

Target image: D1.jpg is classified as: Z with best max_value score: 0.45781180262565613
Target image: D1001.jpg is classified as: Z with best max_value score: 0.5386895537376404
Target image: D101.jpg is classified as: O with best max_value score: 0.6055430769920349
Target image: D1101.jpg is classified as: C with best max_value score: 0.11586544662714005
Target image: D1201.jpg is classified as: Z with best max_value score: 0.49387311935424805
Target image: D1301.jpg is classified as: L with best max_value score: 0.25902923941612244
Target image: D1401.jpg is classified as: V with best max_value score: 0.4841328561306
Target image: D1501.jpg is classified as: Z with best max_value score: 0.35629910230636597
Target image: D1601.jpg is classified as: Y with best max_value score: 0.5442927479743958
Target image: D1701.jpg is classified as: Z with best max_value score: 0.43477359414100647
Target image: D1801.jpg is classified as: Z with best max_value score: 0.46216514706611633
Target ima

Target image: G1001.jpg is classified as: N with best max_value score: 0.4871344268321991
Target image: G101.jpg is classified as: S with best max_value score: 0.3227822482585907
Target image: G1101.jpg is classified as: Q with best max_value score: 0.20298875868320465
Target image: G1201.jpg is classified as: J with best max_value score: 0.5899930000305176
Target image: G1301.jpg is classified as: Y with best max_value score: 0.5385713577270508
Target image: G1401.jpg is classified as: H with best max_value score: 0.4053279757499695
Target image: G1501.jpg is classified as: X with best max_value score: 0.6622999310493469
Target image: G1601.jpg is classified as: P with best max_value score: 0.39713382720947266
Target image: G1701.jpg is classified as: J with best max_value score: 0.49007701873779297
Target image: G1801.jpg is classified as: J with best max_value score: 0.5796324014663696
Target image: G1901.jpg is classified as: J with best max_value score: 0.24817059934139252
Target 

Target image: J1001.jpg is classified as: J with best max_value score: 0.48912182450294495
Target image: J101.jpg is classified as: P with best max_value score: 0.3781581521034241
Target image: J1101.jpg is classified as: P with best max_value score: 0.326760470867157
Target image: J1201.jpg is classified as: X with best max_value score: 0.5232502222061157
Target image: J1301.jpg is classified as: Q with best max_value score: 0.24861779808998108
Target image: J1401.jpg is classified as: J with best max_value score: 0.337027907371521
Target image: J1501.jpg is classified as: X with best max_value score: 0.6502875685691833
Target image: J1601.jpg is classified as: J with best max_value score: 0.5413027405738831
Target image: J1701.jpg is classified as: J with best max_value score: 0.39323338866233826
Target image: J1801.jpg is classified as: M with best max_value score: 0.5148366689682007
Target image: J1901.jpg is classified as: J with best max_value score: 0.4628858268260956
Target ima

Target image: M1001.jpg is classified as: B with best max_value score: 0.30846914649009705
Target image: M101.jpg is classified as: X with best max_value score: 0.5670915842056274
Target image: M1101.jpg is classified as: S with best max_value score: 0.3369917869567871
Target image: M1201.jpg is classified as: E with best max_value score: 0.5447465181350708
Target image: M1301.jpg is classified as: N with best max_value score: 0.38764578104019165
Target image: M1401.jpg is classified as: Q with best max_value score: 0.13655833899974823
Target image: M1501.jpg is classified as: N with best max_value score: 0.3084770441055298
Target image: M1601.jpg is classified as: F with best max_value score: 0.22982215881347656
Target image: M1701.jpg is classified as: J with best max_value score: 0.5212762951850891
Target image: M1801.jpg is classified as: N with best max_value score: 0.4067879617214203
Target image: M1901.jpg is classified as: J with best max_value score: 0.33479416370391846
Target

Target image: P1001.jpg is classified as: J with best max_value score: 0.14003971219062805
Target image: P101.jpg is classified as: P with best max_value score: 0.38464027643203735
Target image: P1101.jpg is classified as: P with best max_value score: 0.303020715713501
Target image: P1201.jpg is classified as: G with best max_value score: 0.4706895351409912
Target image: P1301.jpg is classified as: J with best max_value score: 0.18214084208011627
Target image: P1401.jpg is classified as: P with best max_value score: 0.31259071826934814
Target image: P1501.jpg is classified as: P with best max_value score: 0.3701358139514923
Target image: P1601.jpg is classified as: G with best max_value score: 0.4082981050014496
Target image: P1701.jpg is classified as: O with best max_value score: 0.1849062591791153
Target image: P1801.jpg is classified as: G with best max_value score: 0.30744609236717224
Target image: P1901.jpg is classified as: P with best max_value score: 0.21392226219177246
Target

Target image: S1001.jpg is classified as: P with best max_value score: 0.2702828645706177
Target image: S101.jpg is classified as: P with best max_value score: 0.3908748924732208
Target image: S1101.jpg is classified as: P with best max_value score: 0.2802504897117615
Target image: S1201.jpg is classified as: J with best max_value score: 0.47999393939971924
Target image: S1301.jpg is classified as: Q with best max_value score: 0.13272179663181305
Target image: S1401.jpg is classified as: X with best max_value score: 0.4994323253631592
Target image: S1501.jpg is classified as: Q with best max_value score: 0.11930174380540848
Target image: S1601.jpg is classified as: P with best max_value score: 0.22359800338745117
Target image: S1701.jpg is classified as: P with best max_value score: 0.3452484607696533
Target image: S1801.jpg is classified as: B with best max_value score: 0.18129609525203705
Target image: S1901.jpg is classified as: L with best max_value score: 0.13409379124641418
Targe

Target image: V1001.jpg is classified as: I with best max_value score: 0.2945922315120697
Target image: V101.jpg is classified as: P with best max_value score: 0.33960169553756714
Target image: V1101.jpg is classified as: G with best max_value score: 0.3361406922340393
Target image: V1201.jpg is classified as: Q with best max_value score: 0.19254142045974731
Target image: V1301.jpg is classified as: G with best max_value score: 0.2858736515045166
Target image: V1401.jpg is classified as: I with best max_value score: 0.475771963596344
Target image: V1501.jpg is classified as: P with best max_value score: 0.3010665774345398
Target image: V1601.jpg is classified as: P with best max_value score: 0.29531288146972656
Target image: V1701.jpg is classified as: X with best max_value score: 0.31256207823753357
Target image: V1801.jpg is classified as: X with best max_value score: 0.5251913666725159
Target image: V1901.jpg is classified as: J with best max_value score: 0.387485533952713
Target im

Target image: Y1001.jpg is classified as: P with best max_value score: 0.29875820875167847
Target image: Y101.jpg is classified as: P with best max_value score: 0.31924498081207275
Target image: Y1101.jpg is classified as: P with best max_value score: 0.30000025033950806
Target image: Y1201.jpg is classified as: L with best max_value score: 0.5114603042602539
Target image: Y1301.jpg is classified as: P with best max_value score: 0.34309151768684387
Target image: Y1401.jpg is classified as: K with best max_value score: 0.23682309687137604
Target image: Y1501.jpg is classified as: L with best max_value score: 0.3718358874320984
Target image: Y1601.jpg is classified as: G with best max_value score: 0.3399178087711334
Target image: Y1701.jpg is classified as: S with best max_value score: 0.23687933385372162
Target image: Y1801.jpg is classified as: P with best max_value score: 0.23673415184020996
Target image: Y1901.jpg is classified as: P with best max_value score: 0.3377642035484314
Targ

In [49]:
# HOG descriptor method tanımladık
hog = cv2.HOGDescriptor()

#en iyi accuracye sahip harf dosyasını gostermek icin degisken tanımladık
accuracy_class_score = {}

#target dosyasinda yer alan her bir harf dosyasinin icine giriyoruz
for dir, image_paths in images.items():
    true_class = dir
    total = len(images[dir])
    correct = 0
    folder_best_class = []
    
    #target_images taki harf dosyasındakı resımlerın üzerinden gezmeye baslıyoruz
    for image_path in image_paths:
        target = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)    #binary dondurmeden once grayscale yapıyoruz
        target = cv2.GaussianBlur(target, (7, 7), 0) # resmi bazi yuksek frekanslardan kurtulmamizi, yumusatmamizi sagliyor

        # thresholding ile resmın arka plandan kurtulmak ıcın ben otsu yontemiyle binary aldım dynamıc 
        #resim üzerinde histogram grafigine göre fonkisyon kendisi threshold almıs oldu
      
        target = cv2.threshold(target, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        # resize image ile tüm resimleri aynı boyuta getirdik
        target = cv2.resize(target,  (64, 128), interpolation=cv2.INTER_LINEAR)
        
        #hogs hesaplandık
        target_hog = hog.compute(target)
       
        

        # max degerını bulmak ıcın tum skorları tutmamız lazım bır degısken tanımladık 
        scores = {}
        
        #template imagedaki resimler üzerinde geziyoruz
        for template_name in template_image_name:
            template_path = os.path.join(template_folder,template_name+'.png')
            
            #grayscale cekiyoruz ve gaussıann blur uyguluyoruz
            template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
            template = cv2.GaussianBlur(template, (7, 7), 0) # resmi bazi yuksek frekanslardan kurtulmamizi, yumusatmamizi sagliyor
            
            #otsu ile otomatik threshold hesaplandi ve binary resme cevrildi. threshold altindakiler 0 ustundekiler 255 olarak
            template = cv2.threshold(template, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] 
 
            # resımler aynı boyuta cekilde
            template = cv2.resize(template, (64, 128))
            
            #hogs degeri hesplandı
            template_hog = hog.compute(template)
           
            #cosine similarity hogs degerlerini kullarnark resimlerin benzerlikleri hesaplandı
            scores[template_name] = cosine_similarity(template_hog.reshape(1, -1), target_hog.reshape(1, -1))
            
        #en yüksek scora sahip harfi best_class olarak sakladım
        #Eger best_class true_classa esitse correct sayısnı bir arttır
        best_class = max(scores, key=scores.get)  #scores degerin maksina ait etiketi bulup best_class olarak sakladık
        if true_class == best_class:
            correct += 1
        
        ## targetta yer alan herbir resmin ne ile etiketlendiğini yazdırdım
        
        print("Target image:", os.path.basename(image_path), "is classified as:", best_class,"with best hogs score:", scores[best_class])
        folder_best_class.append(best_class)
        
    # ve harf dosyalarına ait accuracy  hesaplanıp print ettim 
    #bir harf dosyasındaki harflerin yine en cok match olan harfini yazdırdım
    accuracy = correct / total    
    print("Accuracy of ",true_class, "is: ", accuracy)
    print(true_class, 'folder, most matched alphabet is', max(set(folder_best_class), key = folder_best_class.count))
    
    accuracy_class_score[true_class] = accuracy #her bir dosyanın dogruluklarını tutan dictionary

#Tüm dosyalar içinde en yüksek ortlama dogruluk ile sınıflanan dosya
best_accuracy_class = max(accuracy_class_score, key = accuracy_class_score.get)

print('The best classified alphabet is:',best_accuracy_class, 'with accuracy score of ', accuracy_class_score[best_accuracy_class] )

Target image: A1.jpg is classified as: W with best hogs score: [[0.53727716]]
Target image: A1001.jpg is classified as: J with best hogs score: [[0.33697104]]
Target image: A101.jpg is classified as: Y with best hogs score: [[0.48152763]]
Target image: A1101.jpg is classified as: O with best hogs score: [[0.47257403]]
Target image: A1201.jpg is classified as: A with best hogs score: [[0.5843321]]
Target image: A1301.jpg is classified as: C with best hogs score: [[0.40077734]]
Target image: A1401.jpg is classified as: Z with best hogs score: [[0.3967905]]
Target image: A1501.jpg is classified as: D with best hogs score: [[0.39047247]]
Target image: A1601.jpg is classified as: O with best hogs score: [[0.42388046]]
Target image: A1701.jpg is classified as: V with best hogs score: [[0.4958325]]
Target image: A1801.jpg is classified as: C with best hogs score: [[0.4061834]]
Target image: A1901.jpg is classified as: X with best hogs score: [[0.4949811]]
Target image: A2001.jpg is classified

Target image: D201.jpg is classified as: C with best hogs score: [[0.42684436]]
Target image: D2101.jpg is classified as: C with best hogs score: [[0.33288205]]
Target image: D2201.jpg is classified as: C with best hogs score: [[0.33487394]]
Target image: D2301.jpg is classified as: T with best hogs score: [[0.3607925]]
Target image: D2401.jpg is classified as: L with best hogs score: [[0.44747058]]
Target image: D2501.jpg is classified as: H with best hogs score: [[0.22819318]]
Target image: D2601.jpg is classified as: V with best hogs score: [[0.23248863]]
Target image: D2701.jpg is classified as: L with best hogs score: [[0.33338493]]
Target image: D2801.jpg is classified as: S with best hogs score: [[0.34770676]]
Target image: D2901.jpg is classified as: F with best hogs score: [[0.23631358]]
Target image: D301.jpg is classified as: O with best hogs score: [[0.37054926]]
Target image: D401.jpg is classified as: O with best hogs score: [[0.40141696]]
Target image: D501.jpg is classi

Target image: G601.jpg is classified as: H with best hogs score: [[0.36498308]]
Target image: G701.jpg is classified as: G with best hogs score: [[0.34887218]]
Target image: G801.jpg is classified as: N with best hogs score: [[0.3965964]]
Target image: G901.jpg is classified as: H with best hogs score: [[0.33905646]]
Accuracy of  G is:  0.03333333333333333
G folder, most matched alphabet is J
Target image: H1.jpg is classified as: N with best hogs score: [[0.24528007]]
Target image: H1001.jpg is classified as: J with best hogs score: [[0.42173856]]
Target image: H101.jpg is classified as: J with best hogs score: [[0.349594]]
Target image: H1101.jpg is classified as: N with best hogs score: [[0.28504682]]
Target image: H1201.jpg is classified as: J with best hogs score: [[0.42416018]]
Target image: H1301.jpg is classified as: M with best hogs score: [[0.33409232]]
Target image: H1401.jpg is classified as: V with best hogs score: [[0.44420585]]
Target image: H1501.jpg is classified as: M

Target image: K1301.jpg is classified as: W with best hogs score: [[0.5299256]]
Target image: K1401.jpg is classified as: C with best hogs score: [[0.44690612]]
Target image: K1501.jpg is classified as: W with best hogs score: [[0.55421853]]
Target image: K1601.jpg is classified as: W with best hogs score: [[0.443399]]
Target image: K1701.jpg is classified as: C with best hogs score: [[0.41640946]]
Target image: K1801.jpg is classified as: V with best hogs score: [[0.5577478]]
Target image: K1901.jpg is classified as: V with best hogs score: [[0.4991141]]
Target image: K2001.jpg is classified as: L with best hogs score: [[0.467731]]
Target image: K201.jpg is classified as: V with best hogs score: [[0.3952544]]
Target image: K2101.jpg is classified as: M with best hogs score: [[0.3649994]]
Target image: K2201.jpg is classified as: Y with best hogs score: [[0.42866078]]
Target image: K2301.jpg is classified as: M with best hogs score: [[0.34540424]]
Target image: K2401.jpg is classified 

Target image: N2501.jpg is classified as: S with best hogs score: [[0.27027133]]
Target image: N2601.jpg is classified as: S with best hogs score: [[0.33080047]]
Target image: N2701.jpg is classified as: C with best hogs score: [[0.23582423]]
Target image: N2801.jpg is classified as: M with best hogs score: [[0.37236023]]
Target image: N2901.jpg is classified as: N with best hogs score: [[0.30964375]]
Target image: N301.jpg is classified as: Y with best hogs score: [[0.48588043]]
Target image: N401.jpg is classified as: O with best hogs score: [[0.5663878]]
Target image: N501.jpg is classified as: B with best hogs score: [[0.4886669]]
Target image: N601.jpg is classified as: M with best hogs score: [[0.4454763]]
Target image: N701.jpg is classified as: M with best hogs score: [[0.43246502]]
Target image: N801.jpg is classified as: O with best hogs score: [[0.36266854]]
Target image: N901.jpg is classified as: S with best hogs score: [[0.417479]]
Accuracy of  N is:  0.13333333333333333


Target image: Q801.jpg is classified as: A with best hogs score: [[0.21853326]]
Target image: Q901.jpg is classified as: P with best hogs score: [[0.20068319]]
Accuracy of  Q is:  0.13333333333333333
Q folder, most matched alphabet is P
Target image: R1.jpg is classified as: W with best hogs score: [[0.3404976]]
Target image: R1001.jpg is classified as: C with best hogs score: [[0.18781532]]
Target image: R101.jpg is classified as: Y with best hogs score: [[0.21189806]]
Target image: R1101.jpg is classified as: K with best hogs score: [[0.26751333]]
Target image: R1201.jpg is classified as: U with best hogs score: [[0.30254573]]
Target image: R1301.jpg is classified as: W with best hogs score: [[0.2563436]]
Target image: R1401.jpg is classified as: W with best hogs score: [[0.28496283]]
Target image: R1501.jpg is classified as: K with best hogs score: [[0.34011203]]
Target image: R1601.jpg is classified as: J with best hogs score: [[0.26907703]]
Target image: R1701.jpg is classified as

Target image: U1601.jpg is classified as: M with best hogs score: [[0.32540858]]
Target image: U1701.jpg is classified as: N with best hogs score: [[0.31152934]]
Target image: U1801.jpg is classified as: C with best hogs score: [[0.29837605]]
Target image: U1901.jpg is classified as: F with best hogs score: [[0.38959786]]
Target image: U2001.jpg is classified as: Y with best hogs score: [[0.34214914]]
Target image: U201.jpg is classified as: O with best hogs score: [[0.30203617]]
Target image: U2101.jpg is classified as: Y with best hogs score: [[0.33275235]]
Target image: U2201.jpg is classified as: Q with best hogs score: [[0.366337]]
Target image: U2301.jpg is classified as: U with best hogs score: [[0.4488826]]
Target image: U2401.jpg is classified as: D with best hogs score: [[0.4233424]]
Target image: U2501.jpg is classified as: W with best hogs score: [[0.25309062]]
Target image: U2601.jpg is classified as: O with best hogs score: [[0.17904386]]
Target image: U2701.jpg is classi

Target image: X2501.jpg is classified as: O with best hogs score: [[0.21458447]]
Target image: X2601.jpg is classified as: O with best hogs score: [[0.21014896]]
Target image: X2701.jpg is classified as: O with best hogs score: [[0.28175545]]
Target image: X2801.jpg is classified as: F with best hogs score: [[0.3513461]]
Target image: X2901.jpg is classified as: O with best hogs score: [[0.3016819]]
Target image: X301.jpg is classified as: T with best hogs score: [[0.2545367]]
Target image: X401.jpg is classified as: Q with best hogs score: [[0.16327244]]
Target image: X501.jpg is classified as: G with best hogs score: [[0.24739762]]
Target image: X601.jpg is classified as: F with best hogs score: [[0.24048182]]
Target image: X701.jpg is classified as: X with best hogs score: [[0.35310116]]
Target image: X801.jpg is classified as: S with best hogs score: [[0.19477464]]
Target image: X901.jpg is classified as: L with best hogs score: [[0.2297905]]
Accuracy of  X is:  0.03333333333333333